<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0625/%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%98%88%EC%A0%9C(commit_no).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00


In [2]:
# hugging face transformers 라이브러리 설치
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00


In [2]:
# core
import os, shutil, unicodedata
import pickle, re
import numpy as np
import pandas as pd

# visualize
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm  # 폰트
from matplotlib import rc # 폰트
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# tensorflow
import tensorflow as tf
## keras
from tensorflow.keras import layers, models, metrics, callbacks
## layers & models
from tensorflow.keras.layers import Layer, Input, Embedding, Concatenate, Flatten, Normalization
from tensorflow.keras.layers import Dense, LSTM, GRU, Attention
from tensorflow.keras.models import Sequential, Model
## optimizers & callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
## preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
## utils
from tensorflow.keras.utils import plot_model
## saving
from tensorflow.keras.saving import save_model

# Huggingface
from transformers import TFBertModel, BertForMaskedLM, FillMaskPipeline, TFBertForNextSentencePrediction, TFRobertaModel, AutoModel, TFBertForSequenceClassification

from transformers import TFBertTokenizer, AutoTokenizer, AutoConfig
from transformers import TFGPT2LMHeadModel
# scikit-learn
## models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer

# Time Check
import time
from tqdm.notebook import tqdm
tqdm.pandas()

# customization
## font
# plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = True

## preprocessed Data 불러오기

In [3]:
# # 데이터 불러오기
# import pandas as pd
# import pymysql

# # Connect to the MariaDB database
# conn = pymysql.connect(
#     host='private.dotge.site',
#     user='ade345',
#     password='dbslwms123',
#     database='books'
# )
# cursor = conn.cursor()
# # Execute the SQL query to fetch the data
# sql = "SELECT * FROM devcon" # SELECT * FROM devcon_test : test 파일
# cursor.execute(sql)
# # Fetch all the rows returned by the query
# rows = cursor.fetchall()
# data = []
# # Iterate over the rows and append data to the list as dictionaries
# for row in rows:
#     raw_id = row[1]
#     raw_first_party = row[2]
#     raw_second_party = row[3]
#     raw_facts = row[4]
#     raw_winner = row[5]
#     # Create a dictionary for the row
#     row_dict = {
#         '_id': raw_id,
#         'raw_first_party': raw_first_party,
#         'raw_second_party': raw_second_party,
#         'raw_facts': raw_facts,
#         'raw_winner': raw_winner
#     }
#     # Append the dictionary to the data list
#     data.append(row_dict)
# # Create a DataFrame from the data list
# df = pd.DataFrame(data)
# # Close the cursor and connection
# cursor.close()
# conn.close()
# # Print the DataFrame
# df

In [4]:
df = pd.read_csv('/content/preprocessed_1.csv')
df

,Unnamed: 0,ID,first_party,second_party,facts,first_party_winner,facts_split
0,0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1,"['On', 'June', '27,', '1962,', 'Phil', 'St.', ..."
1,1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0,"['Ramon', 'Nelson', 'was', 'riding', 'his', 'b..."
2,2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1,"['An', 'Alabama', 'state', 'court', 'convicted..."
3,3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0,"['Victor', 'Linkletter', 'was', 'convicted', '..."
4,4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1,"['On', 'April', '24,', '1953', 'in', 'Selma,',..."
...,...,...,...,...,...,...,...
2473,2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1,"['Congress', 'amended', 'the', 'Clean', 'Air',..."
2474,2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1,"['Alliance', 'Bond', 'Fund,', 'Inc.,', 'an', '..."
2475,2475,TRAIN_2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0,"['In', '1992,', 'the', 'District', 'Court', 's..."
2476,2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0,"['On', 'March', '8,', '1996,', 'Enrico', 'St.'..."


## 모델 불러오기

In [5]:
# nlpaueb/legal-bert-small-uncased
tokenizer_3 = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model_3 = TFBertModel.from_pretrained("nlpaueb/legal-bert-base-uncased")
config_3 = AutoConfig.from_pretrained("nlpaueb/legal-bert-base-uncased") # 모델 구조 파악(레이어는 몰라도 속성 알기 좋아서 썼습니다)

config_3

Some layers from the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


BertConfig {
  "_name_or_path": "nlpaueb/legal-bert-base-uncased",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [6]:
# 토큰화
token_3 = tokenizer_3(df['facts'].tolist(), truncation=True, padding=True, return_tensors = 'tf')

# 토큰화 결과 체크([PAD] 토큰 제외)
print(f'''token_3의 0번 문장:\n{[token for token in token_3[0].tokens if not token == '[PAD]']}''')

token_3의 0번 문장:
['[CLS]', 'on', 'june', '27', ',', '1962', ',', 'phil', 'st', '.', 'am', '##ant', ',', 'a', 'candidate', 'for', 'public', 'office', ',', 'made', 'a', 'television', 'speech', 'in', 'baton', 'roug', '##e', ',', 'louisiana', '.', 'during', 'this', 'speech', ',', 'st', '.', 'am', '##ant', 'accused', 'his', 'political', 'opponent', 'of', 'being', 'a', 'communist', 'and', 'of', 'being', 'involved', 'in', 'criminal', 'activities', 'with', 'the', 'head', 'of', 'the', 'local', 'teamster', '##s', 'union', '.', 'finally', ',', 'st', '.', 'am', '##ant', 'implicated', 'herman', 'thompson', ',', 'an', 'east', 'baton', 'roug', '##e', 'deputy', 'sheriff', ',', 'in', 'a', 'scheme', 'to', 'move', 'money', 'between', 'the', 'teamster', '##s', 'union', 'and', 'st', '.', 'am', '##ant', '[UNK]', 's', 'political', 'opponent', '.', 'thompson', 'successfully', 'sued', 'st', '.', 'am', '##ant', 'for', 'defam', '##ation', '.', 'louisiana', '[UNK]', 's', 'first', 'circuit', 'court', 'of', 'appeals

In [7]:
# X, y 지정
X_data = token_3
y_data = df['first_party_winner'].astype(np.int32) # 그냥 astype() 좋아해서 쓴것입니다. int()써도 문제 없을거에요

In [8]:
# Dataset 타입으로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_data),
    y_data
))

### 모델링

In [ ]:
# 모델 수정
class MyModel(tf.keras.Model):
    def __init__(self, model_name):
        super().__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt = True) ## from_tf -> from_pt ##
        self.classifier = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')

    def call(self, input_ids = None, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction


# 모델 인스턴스
model_name = "nlpaueb/legal-bert-base-uncased"
model = MyModel(model_name)


# 컴파일optimizer=Adam(learning_rate=1e4)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e4)
loss = tf.keras.losses.BinaryCrossentropy() ## SparseCategoricalCrossenctropy -> Binary ##

model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

num_train = 1000

# 학습
model.fit(
    train_dataset.shuffle(num_train).batch(16),
    epochs=2, batch_size=16,
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

Epoch 1/2
